**Задание 3**  

В пакете для третьего задания три файла:
В файле ***Tariff_plans_change.csv*** содержатся выборочные данные о транзакциях, связанных с подключениями и отключениями тарифных планов абонентами, которые меняли тарифные планы в первом полугодии 2017 года: 

SUBSCRIBER_ID	 -  условный идентификатор абонента

TARIFF_PLAN_ID – условный идентификатор тарифного плана	

START_DTTM	 - дата и время подключения плана

END_DTTM – дата и время отключения (значение $null$ означает, что план актуален на текущий момент).

В файле ***Charges.csv*** содержатся помесячные исторические данные об общих расходах абонентов на мобильную связь:

SUBSCRIBER_ID	 -  условный идентификатор абонента

BILL_MONTH – биллинговый период в формате YYYY-MM-01 (например, строки со значением ‘2016-01-01’ содержат данные о расходах и потреблении за январь 2016 года)

CHARGES – общая сумма  счета за услуги связи

В файле ***Suspended.csv*** содержатся исторические данные о блокировках абонентов в транзакционной форме:

SUBSCRIBER_ID	 -  условный идентификатор абонента

START_DT - дата начала блокировки

END_DT – дата окончания блокировки (значение $null$ означает, что абонент остается блокированным на текущий момент).

**Необходимо исследовать следующие вопросы:**

1)	Направления смен тарифных планов: с каких тарифных планов и на какие шли  наибольшие перетоки? Визуализировать потоки миграций на диаграмме. 

2)	Насколько изменился среднемесячный счет абонентов за период 3 месяца после месяца смены тарифного плана в сравнении с периодом 3 месяца до месяца смены тарифного плана. Какие направления смен тарифных планов характеризовались ростом среднего счета в трехмесячном периоде, а какие – сокращением.  Визуализировать результаты на диаграммах.

3)	Аналогично п.2, только относительно изменения уровня блокировок: насколько реже или чаще стали попадать в блокировки абоненты после миграции в целом и для каждого направления миграции в отдельности. Для сравнения использовать те же периоды: 3 месяца до месяца смены плана и 3 месяца после месяца смены.


In [2]:
import pandas as pd
import seaborn as sns

In [3]:
tariff_plans_change = pd.read_csv("Tariff_plans_change.csv")
charges = pd.read_csv("Charges.csv")
suspended = pd.read_csv("Suspended.csv")

In [8]:
#Приведем все названия столбцов к нижнему регистру
ariff_plans_change.columns = tariff_plans_change.columns.str.lower()

**Направление смен тарифных планов**

In [18]:
changes = sorted(tariff_plans_change['tariff_plan_id'].unique())
print(f"Уникальные тарифные планы:", changes)

Уникальные тарифные планы: [1, 2, 3, 4, 5]


При выборе k элементов из n с возвращением и с учетом порядка, общее количество различных наборов составит n^k.  Но мы не должны учитывать, когда оба выбранных элемента в нашем случае равны. Таким образом, получится 20 пар.

In [22]:
changes_pair = []
for i in changes:
    for j in changes:
        if i != j:
            changes_pair.append((i,j))

In [27]:
print("Возможные перетоки тарифных планов:")
for i in changes_pair:
    print(f"C {i[0]} тарифного плана на {i[1]}")

Возможные перетоки тарифных планов:
C 1 тарифного плана на 2
C 1 тарифного плана на 3
C 1 тарифного плана на 4
C 1 тарифного плана на 5
C 2 тарифного плана на 1
C 2 тарифного плана на 3
C 2 тарифного плана на 4
C 2 тарифного плана на 5
C 3 тарифного плана на 1
C 3 тарифного плана на 2
C 3 тарифного плана на 4
C 3 тарифного плана на 5
C 4 тарифного плана на 1
C 4 тарифного плана на 2
C 4 тарифного плана на 3
C 4 тарифного плана на 5
C 5 тарифного плана на 1
C 5 тарифного плана на 2
C 5 тарифного плана на 3
C 5 тарифного плана на 4
